In [1]:
import jieba
import math
import os
import json
import jieba.posseg as pseg

from collections import OrderedDict

In [2]:
class TF_IDF():
    def __init__(self):
        self.docs = {}
        self.seg_docs = self.get_seg_docs()
        self.stopword = []
        self.tf = []
        self.df = {}
        self.idf = {}
        self.topK_idf = {}
        self.bow = {}
        self.cal_tfidf()

    def read_file(self, path, type):
        if type == 'json':
            with open(path, 'r', encoding='utf-8') as file:
                data = json.loads(file.read())
        elif type == 'txt':
            with open(path, 'r', encoding='utf-8') as file:
                data = file.read()
        return data

    def get_seg_docs(self):
        _seg_docs = []
        FOLDER_NAME = 'data_tfidf'
        DOCUMENT = 'data_4125_512.json'
        STOPWORD = 'stopword.txt'
        #FILE_DIR = os.path.join(os.path.split(os.path.realpath(__file__))[0], FOLDER_NAME)

        self.docs = self.read_file('.\\' + FOLDER_NAME +'\\' + DOCUMENT, 'json')
        self.stopword = self.read_file('.\\' + FOLDER_NAME + '\\' + STOPWORD, 'txt')
        for i in range(len(self.docs)):
            content_seg = [w for w in jieba.lcut(self.docs[i]['content']) if len(w) > 1 and w not in self.stopword and w.isalpha()]
            _seg_docs.append(content_seg)
        return _seg_docs
    """
    計算tf,idf結果
    tf:[{word1:3,word2:4,word4:2},{word2:5,word3:7, word4:2},{....},.......]
    df:{word1:6個文檔,word2:3個文檔,word3:5個文檔,word4:4個文檔......}
    idf:{word1:idf(word1),word2:idf(word2),word3:idf(word3)..........}
    """
    def cal_tfidf(self):
        for doc in self.seg_docs:
            bow = {}
            for word in doc:
                if not word in bow:
                    bow[word] = 0
                bow[word] += 1
            self.tf.append(bow)
            for word, _ in bow.items():
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1
        for word, df in self.df.items():
            #只出現過在一篇文檔的詞不要(選擇性)
            if df < 2:
                pass
            else:
                self.idf[word] = math.log10(len(self.seg_docs) / df)

    def tf(self, index, word):
        return self.tf[index][word]

    def idf(self, word):
        return self.idf[word]

    def tf_idf(self, index, word):
        return self.tf[index][word]*self.idf[word]

    def get_topK_idf(self, k, reverse = True):
        self.topK_idf = OrderedDict(sorted(self.idf.items(), key=lambda t: t[1], reverse = reverse)[:k])
        return  self.topK_idf

    def get_docment(self):
        return self.docs

    def get_title(self, index):
        return self.docs[index]['title']

    def get_content(self, index):
        return self.docs[index]['content']

    def set_bag_of_word(self, bow):
        self.bow = bow

    def get_text_vector(self, index):
        return [1*self.tf_idf(index, w) if w in jieba.lcut(self.docs[index]['content']) else 0 for w in self.bow]

    def cosine_similarity(self, v1, v2):
        #compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)
        sum_xx, sum_xy, sum_yy = 0.0, 0.0, 0.0
        for i in range(0, len(v1)):
            x, y = v1[i], v2[i]
            sum_xx += math.pow(x, 2)
            sum_yy += math.pow(y, 2)
            sum_xy += x * y
        try:
            return sum_xy / math.sqrt(sum_xx * sum_yy)
        except ZeroDivisionError:
            return 0

In [3]:
tf_idf = TF_IDF()
topK = tf_idf.get_topK_idf(50000, True)
#保存bag of word
tf_idf.set_bag_of_word(set(topK.keys()))
#得到文章第1篇跟第11篇的向量
vec1 = tf_idf.get_text_vector(0)
vec2 = tf_idf.get_text_vector(10)
#計算文件與文件的cosine similarity
score1 = tf_idf.cosine_similarity(vec1, vec1)
score2 = tf_idf.cosine_similarity(vec1, vec2)
#print(topK, score1, score2)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 0.436 seconds.
Prefix dict has been built succesfully.


In [4]:
f = open("data_tfidf\\result.csv", "a", encoding='UTF-8')
f.write("詞,詞性,P\n")

for k, v in topK.items():
    try:
        _k = str(k)
        words = pseg.cut(_k)
        n = ""
        for _w in words:
            n = _w.flag
        s = _k + "," + n + "," + str(v) + "\n"
        f.write(s)
    except Exception as e:
        print(e)
        continue

f.close()

In [5]:
print(tf_idf.get_content(0))

邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃園分局埔子派出所內之值班臺，明知員警徐芳振係依法執行職務之公務員，因不滿員警告以所詢事項應向其他單位辦理，竟基於妨害公務員執行職務之犯意，至停於於派出所外之車輛內取出木盒，並返回派出所內，持該木盒敲打值班臺，而以此對物之強暴方式，妨害徐芳振執行公務。


In [6]:
#m 數字 t 時間 c 連詞
#regex
#/○{2,}/
#/\d{10}/
#/[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}/ 車牌
import jieba.posseg as pseg

In [7]:
words =pseg.cut(tf_idf.get_content(1500))
for w in words:
    print(w.word,w.flag)
    break


被告 n


In [8]:
import numpy as np
import pandas as pd
import re
from datetime import datetime

#my_regex = "\（.*?\）|\(([^\)0-9零一二三四五六七八九十]?|[^\).]{2,})\)"

#tmpstr = '丙○○意圖為自己不法之所有，(甲)YY(假)XX(十一)(一)a(一一)b(一一一)c(年一)d(一年一)e(一一一年)A(9年)B(99年)C(999年)A(AAA)於民國105年10月8日晚間7時20分許，騎乘登記於不知情之王介平名下之車牌號碼000-000號普通重型機車，行經高雄市○○區○○○路000號前時，將口罩脫下遮掩車牌，自後靠近步行在人行道上之甲○○，趁甲○○不及防備之際，以左手徒手掠取甲○○右手持有之背包1只（內有現金新臺幣【下同】9500元、索尼牌ZR綠色及Xperformance金色行動電話共2具【IMEI：000000000000000、000000000000000號】、長夾1個、信用卡及金融卡共5張、國民身分證2張、國民健康保險卡1張、普通重型機車駕照1張、識別證2張等物）得手後逃逸。嗣因丙○○將背包內之現金取走後，將背包棄置在高雄市鳳山區武營路某處，再經民眾在高雄市鳳山區衛武營附近拾獲上開背包（內有長夾1個、國民身分證2張、國民健康保險卡1張、信用卡3張、金融卡1張等物），遂送交警察機關發還甲○○領回。(二)於同年10月10日晚間11時5分許，騎乘向不知情之劉伊恩借用之車牌號碼000-0000號普通重型機車，行經高雄市○○區○○○路000號前時，使用口罩遮掩車牌，自後靠近步行在人行道上之少年乙○○（89年生，基於兒童及少年福利與權益保障法第69條第2項規定不得揭露足以識別少年身分之資訊，年籍詳卷），趁乙○○不及防備之際，以左手徒手掠取乙○○右手持有之背包1只（內有皮夾1只、現金3萬2000元、提款卡1張、國民身分證1張、國民健康保險卡1張等物）得手後逃逸，並將現金取出，其餘物品則丟棄。嗣為警循線於105年10月16日凌晨2時許，在高雄市○○區○○路000巷00號前查獲，並扣得現金2600元。'
#print((tmpstr))
#print(len(tmpstr))
#print((re.sub(my_regex, "", tmpstr)))
#print(len(re.sub(my_regex, "", tmpstr)))

In [9]:
dt_idf = pd.read_csv("data_tfidf\\result.csv", encoding='utf8')
dt_idf = dt_idf[dt_idf["P"] > 1].drop(columns=['詞性']).set_index('詞').T.to_dict('list')
dt_idf['豐樂']

[3.3143939572219625]

In [10]:
df=pd.read_csv('..//STBert//data//Result_Sbert.csv', index_col=0)
#df=pd.read_csv('.//data//Result_R5.csv', index_col=0)
df.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...
2,臺灣臺中地方法院,2011-01-25T00:00:00+08:00,"100,中交簡,100",刑事,公共危險,判決,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,臺灣臺中地方法院刑事簡易判決100年度中交簡字第100號聲請人臺灣臺中地方法院檢察署檢察官被...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...


In [11]:
df['reason'] = df['reason'].str.replace('等','')
df['reason'] = df['reason'].str.replace('罪','')
df['reason'] = df['reason'].str.replace("條例","")
df['reason'] = df['reason'].str.replace("防制","")
df['reason'] = df['reason'].str.replace("管制","")
df['reason'] = df['reason'].str.replace("違反","")
df.loc[:,'reason'].value_counts()

公共危險      347
毀損        329
妨害公務      320
妨害名譽      308
妨害自由      266
傷害        242
侵占        231
搶奪        220
過失傷害      216
竊盜        204
殺人未遂      202
妨害風化      190
著作權法      190
偽造文書      171
妨害性自主     168
詐欺        159
強盜        141
毒品危害       65
毀棄損壞       38
殺人         37
過失致死        7
貪污          6
賭博          6
槍砲彈藥刀械      5
醫師法         3
商標法         2
藥事法         2
替代役實施       1
偽證          1
Name: reason, dtype: int64

In [12]:
my_regex = r"[\(\（]([^一二三四五六七八九零十1234567890]{1}[^\)\）]+|[一二三四五六七八九零十1234567890]{1}[^\)\）]{1,})[\)\）]"
df['Situation'] = df['Situation'].str.replace(my_regex, "")
#/○{2,}/
regex1 = r"/○{2,}"
df['Situation'] = df['Situation'].str.replace(regex1, "")

#/\d{10}/
regex2 = r"\d{10}"
df['Situation'] = df['Situation'].str.replace(regex2, "")

#/[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}/ 車牌
regex3 = r"[A-Z0-9]{2,3}\-[A-Z0-9]{3,4}"
df['Situation'] = df['Situation'].str.replace(regex3, "")

In [13]:
run = 0
for index, row in df.iterrows():
    try:
        words =pseg.cut(row.Situation)
        seq = ''
        for w, l in words:
            if (w in dt_idf):
                continue
            elif (l in ('m', 't', 'c')):
                continue
            else:
                seq += w
        df.loc[index, 'Situation'] = seq
        run += 1
        if (run % 1000 == 0):
            today = datetime.now()
            print(run, ' => ', today)
    except:
        continue

df.head(2)

1000  =>  2020-11-04 23:35:49.790867
2000  =>  2020-11-04 23:37:27.311297
3000  =>  2020-11-04 23:39:11.937401
4000  =>  2020-11-04 23:40:39.760021


,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國時分許，在桃園市○○區○○路桃園市埔子內之臺，明知員警徐芳振係執行之，因不滿員警...
2,臺灣臺中地方法院,2011-01-25T00:00:00+08:00,"100,中交簡,100",刑事,公共危險,判決,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,臺灣臺中地方法院刑事簡易判決100年度中交簡字第100號聲請人臺灣臺中地方法院檢察署檢察官被...,林耀明於民國時許時，在臺中市○○區○○路與路之，與後，已達不能駕駛之，猶於同時許，駕駛車牌號...


In [14]:
df.to_csv("data_tfidf\\result1.csv", index=0)